PowerShell 7 has some new pipeline operators which which can reduce the amount of code that needs to be written.

Here is a manufactured example. Lets get the content of a log file for processing

In [41]:
$dockerlog = "$env:LOCALAPPDATA\Docker\log.txt"

$dockerlogcontent = Get-Content $dockerlog
$dockerlogcontent[0]

Version: 3.0.0 (50684)


If the file doesnt exist we will get two errors, one for the file not existing and one because the array is null so we cannot access the first item.

In [42]:
$dockerlog = "$env:LOCALAPPDATA\Docker\1og.txt"

$dockerlogcontent =  Get-Content $dockerlog
$dockerlogcontent[0]

Get-Content: 


Line |


   3 |  $dockerlogcontent =  Get-Content $dockerlog


     |                       ~~~~~~~~~~~~~~~~~~~~~~


     | Cannot find path 'C:\Users\mrrob\AppData\Local\Docker\1og.txt' because it does not exist.


InvalidOperation: 


Line |


   4 |  $dockerlogcontent[0]


     |  ~~~~~~~~~~~~~~~~~~~~


     | Cannot index into a null array.


We can wrap this to ensure that the user doesnt receive these errors

In [43]:
$dockerlog = "$env:LOCALAPPDATA\Docker\1og.txt"
try {
    $dockerlogcontent =  Get-Content $dockerlog
    $dockerlogcontent[0]
}
catch {
    Write-Warning "Uh-Oh. The Beard is sad"
}



Get-Content: 


Line |


   3 |      $dockerlogcontent =  Get-Content $dockerlog


     |                           ~~~~~~~~~~~~~~~~~~~~~~


     | Cannot find path 'C:\Users\mrrob\AppData\Local\Docker\1og.txt' because it does not exist.


Ah we need a terminating error to hit the catch

In [44]:
$ErrorActionPreference = 'Stop' 
$dockerlog = "$env:LOCALAPPDATA\Docker\1og.txt"
try {
    $dockerlogcontent =  Get-Content $dockerlog
    $dockerlogcontent[0]
}
catch {
    Write-Warning "Uh-Oh. The Beard is sad"
}
$ErrorActionPreference = 'Continue' 

or we could use `Test-Path` in an if statement

In [45]:
$ErrorActionPreference = 'Stop' 
$dockerlog = "$env:LOCALAPPDATA\Docker\1og.txt"
if(Test-Path $dockerlog){
    try {
        $dockerlogcontent =  Get-Content $dockerlog
        $dockerlogcontent[0]
    }
    catch {
        Write-Warning "Uh-Oh. The Beard is sad"
    }
} else {
    Write-Warning "Uh-Oh. The Beard is sad - that file doesnt exist"
}

$ErrorActionPreference = 'Continue' 

With the new pipeline operators, we can make that easier to read and write less code.

First command fails, causing second not to be executed

In [46]:
$dockerlog = "$env:LOCALAPPDATA\Docker\1og.txt"

Get-ChildItem $dockerlog -ErrorAction SilentlyContinue && Get-Content $dockerlog 


and if we want a message as well we could use the `||` pipeline operator

First command fails, so the second command is executed

In [47]:
$dockerlog = "$env:LOCALAPPDATA\Docker\1og.txt"

Get-Content $dockerlog -ErrorAction SilentlyContinue || Write-Warning "Uh-Oh. The Beard is sad"


and if we want to assign that to a variable then we can

In [52]:
$ErrorActionPreference = 'SilentlyContinue'
# File does not exist
$dockerlog = "$env:LOCALAPPDATA\Docker\1og.txt"
$dockerlogcontent = Get-Content $dockerlog -ErrorAction SilentlyContinue  || Write-Warning "Uh-Oh. The Beard is sad"
$dockerlogcontent[0] || Write-Warning "Uh-Oh. The Beard is sad - We have no content"

#File Exists
$dockerlog = "$env:LOCALAPPDATA\Docker\log.txt"
$dockerlogcontent = Get-Content $dockerlog || Write-Warning "Uh-Oh. The Beard is sad"
$dockerlogcontent[0] || Write-Warning "Uh-Oh. The Beard is sad - We have no content"

$ErrorActionPreference = 'Continue'


Version: 3.0.0 (50684)


Nulls

?? 

The best way to describe this is that your output will be the value on the left hand side of the operator if it isn't null, but if it is null then it will output whatever is on the right of the operator.[Thanks](https://toastit.dev/2020/03/11/ps7now-null-conditional/)

First, we will do this in a couple of Windows PowerShell equivalent ways

In [3]:
$cleanShaven = $null

if($null -eq $cleanShaven){
    "No Beard Found!"
} else {
    $cleanShaven
}

if(-not $cleanShaven){
    "No Beard Found!"
} else {
    $cleanShaven
}


No Beard Found!


No Beard Found!


In [2]:
$cleanShaven = $null
$cleanShaven ?? "No Beard Found!"

No Beard Found!


You can even use commands

In [4]:
$cleanShaven = $null
$cleanShaven ?? ((Get-ChildItem C:\temp).Count)

99


**BUT** Remember this is evaluating for `$null`

In [5]:
$cleanShaven = $false
$cleanShaven ?? ((Get-ChildItem C:\temp).Count)

False


In [6]:
$cleanShaven = Test-Path C:\notthere
$cleanShaven ?? ((Get-ChildItem C:\temp).Count)

False


In [9]:
(Get-DbaDatabase -SqlInstance Beard-Desktop -Database Nope) ?? (Write-Warning "No Database")

Also remember that a NULL from a SQL query is not the same as a `$null`

In [35]:
$Query = "SELECT TOP 1 NULL AS Beard FROM sys.databases"
$DbResults = (Invoke-DbaQuery -SqlInstance Beard-Desktop -Query $Query).Beard
$DbResults -is [System.DBNull]
$null -eq $DbResults  

True


False


In [36]:
$Query = "SELECT TOP 1 NULL AS Beard FROM sys.databases"
$ItsANull = (Invoke-DbaQuery -SqlInstance Beard-Desktop -Query $Query).Beard
$ItsANull ?? (Write-Warning "I will output if I am NULL")

In [37]:
$Query = "SELECT TOP 1 NULL AS Beard FROM sys.databases"
$ItsANull = (Invoke-DbaQuery -SqlInstance Beard-Desktop -Query $Query).Beard
if($ItsANull -is [System.DBNull]){
    $ItsANull = $null
}
$ItsANull ?? (Write-Warning "I will output if I am NULL")